In [1]:
import time
import math, os

from coffea.nanoevents import BaseSchema

import awkward as ak
import numpy as np
from coffea import processor

from hist import Hist

from coffea.nanoevents.methods import candidate
ak.behavior.update(candidate.behavior)

import matplotlib.pyplot as plt
from plotter import create_plot1d, create_plot2d
from utils import *

In [12]:
muon_cols = ['Muon_charge', 'Muon_dxy', 'Muon_dxyErr', 'Muon_dz', 'Muon_dzErr', 'Muon_eta', 'Muon_isGlobal', 'Muon_mass',
             'Muon_phi', 'Muon_pt', 'Muon_ptErr', 'Muon_softId', 'Muon_vtxIdx', 'Muon_vtxFlag', 'Muon_simIdx', 'Muon_x',
             'Muon_y', 'Muon_z',]

dimu_cols = ['Dimu_pt', 'Dimu_eta', 'Dimu_phi', 'Dimu_rap', 'Dimu_mass', 'Dimu_charge', 'Dimu_vtxIdx', 'Dimu_chi2', 'Dimu_dl', 
             'Dimu_dlErr', 'Dimu_dlSig', 'Dimu_cosphi', 'Dimu_x', 'Dimu_y', 'Dimu_z', 'Dimu_t1muIdx', 'Dimu_t2muIdx',]

dimu_cols_cov = ['Dimu_Covxx',	'Dimu_Covyx', 'Dimu_Covyy', 'Dimu_Covzx', 'Dimu_Covzy', 'Dimu_Covzz' 'Dimu_pt', 'Dimu_eta', 
                 'Dimu_phi', 'Dimu_rap', 'Dimu_mass', 'Dimu_charge', 'Dimu_vtxIdx', 'Dimu_chi2', 'Dimu_dl', 'Dimu_dlErr', 
                 'Dimu_dlSig', 'Dimu_cosphi', 'Dimu_x', 'Dimu_y', 'Dimu_z', 'Dimu_t1muIdx', 'Dimu_t2muIdx',]

d0_cols = ['D0_pt', 'D0_eta', 'D0_phi', 'D0_rap', 'D0_mass12', 'D0_mass21', 'D0_vtxIdx', 'D0_chi2', 'D0_dl', 'D0_dlErr', 
           'D0_dlSig', 'D0_cosphi', 'D0_x', 'D0_y', 'D0_z', 'D0_hasMuon',
           'D0_t1pt', 'D0_t1eta', 'D0_t1phi', 'D0_t1chindof', 'D0_t1nValid', 'D0_t1nPix', 'D0_t1dxy', 'D0_t1dz', 'D0_t1chg', 
           'D0_t2pt', 'D0_t2eta', 'D0_t2phi', 'D0_t2chindof', 'D0_t2nValid', 'D0_t2nPix', 'D0_t2dxy', 'D0_t2dz', 'D0_t2chg',]

d0_cols_cov = ['D0_Covxx', 'D0_Covyx', 'D0_Covyy', 'D0_Covzx', 'D0_Covzy', 'D0_Covzz', 'D0_pt', 'D0_eta', 'D0_phi', 'D0_rap', 
               'D0_mass12', 'D0_mass21', 'D0_vtxIdx', 'D0_chi2', 'D0_dl', 'D0_dlErr', 'D0_dlSig', 'D0_cosphi', 'D0_x', 'D0_y', 
               'D0_z', 'D0_hasMuon',
               'D0_t1pt', 'D0_t1eta', 'D0_t1phi', 'D0_t1chindof', 'D0_t1nValid', 'D0_t1nPix', 'D0_t1dxy', 'D0_t1dz', 'D0_t1chg', 
               'D0_t2pt', 'D0_t2eta', 'D0_t2phi', 'D0_t2chindof', 'D0_t2nValid', 'D0_t2nPix', 'D0_t2dxy', 'D0_t2dz', 'D0_t2chg',]

dstar_cols = ['Dstar_pt', 'Dstar_eta', 'Dstar_phi', 'Dstar_rap', 'Dstar_deltam', 'Dstar_deltamr', 'Dstar_vtxIdx', 'Dstar_hasMuon',
              'Dstar_D0pt', 'Dstar_D0eta', 'Dstar_D0phi', 'Dstar_D0mass', 'Dstar_D0chi2', 'Dstar_D0dl', 'Dstar_D0dlErr',
              'Dstar_D0dlSig', 'Dstar_D0cosphi', 'Dstar_D0x', 'Dstar_D0y', 'Dstar_D0z',
              'Dstar_Kpt', 'Dstar_Keta', 'Dstar_Kphi', 'Dstar_KvtxIdx', 'Dstar_Kchindof', 'Dstar_KnValid', 'Dstar_KnPix', 
              'Dstar_Kdxy', 'Dstar_Kdz', 'Dstar_Kchg',
              'Dstar_pipt', 'Dstar_pieta', 'Dstar_piphi', 'Dstar_pivtxIdx', 'Dstar_pichindof', 'Dstar_pinValid', 'Dstar_pinPix',
              'Dstar_pidxy', 'Dstar_pidz', 'Dstar_pichg',
              'Dstar_pispt', 'Dstar_piseta', 'Dstar_pisphi', 'Dstar_pisvtxIdx', 'Dstar_pischindof', 'Dstar_pisnValid', 'Dstar_pisnPix',
              'Dstar_pisdxy', 'Dstar_pisdz', 'Dstar_simIdx', 'Dstar_D0simIdx', 'Dstar_pischg',]

gen_part_cols = ['GenPart_eta', 'GenPart_genPartIdxMother', 'GenPart_mass', 'GenPart_pdgId', "GenPart_phi", "GenPart_pt", 
                 'GenPart_status', 'GenPart_Id', 'GenPart_parpdgId', 'GenPart_sparpdgId', 'GenPart_numberOfDaughters', 
                 'GenPart_nstchgdaug', 'GenPart_vx', 'GenPart_vy', 'GenPart_vz', 'GenPart_mvx', 'GenPart_mvy', 'GenPart_mvz', 
                 'GenPart_recIdx',]

pvtx_cols = ['PVtx_Covxx',	'PVtx_Covyx', 'PVtx_Covyy', 'PVtx_Covzx', 'PVtx_Covzy', 'PVtx_Covzz', 'PVtx_isGood', 'PVtx_x', 
             'PVtx_y', 'PVtx_z', 'PVtx_Id', 'PVtx_sumPt', 'PVtx_ntrk',]

D0_PDG_MASS = 1.864

def fill_hists(hists, candidate, get_deltam=False):
    hists['pt'].fill(pt=ak.flatten(candidate.pt))
    hists['eta'].fill(eta=ak.flatten(candidate.eta))
    hists['phi'].fill(phi=ak.flatten(candidate.phi))
    if get_deltam:
        if 'Gen' in hists['deltam'].axes[0].label:
            hists['deltam'].fill(deltam=np.repeat(0.145, len(ak.flatten(candidate))))
        else:
            hists['deltam'].fill(deltam=ak.flatten(candidate.deltam))
    else:
        hists['mass'].fill(mass=ak.flatten(candidate.mass))

In [18]:
""" paths = ['/eos/user/k/kmotaama/CRAB_UserFiles/UpsilonToMuMuDstarToD0pi/220120_145243/0000',
         '/eos/user/k/kmotaama/CRAB_UserFiles/UpsilonToMuMuDstarToD0pi/220120_145243/0001'] """
paths = ['/eos/user/k/kmotaama/CRAB_UserFiles/UpsilonToMuMuDstarToD0pi/220127_103512/0000',
         '/eos/user/k/kmotaama/CRAB_UserFiles/UpsilonToMuMuDstarToD0pi/220127_103512/0001']

files = get_root_files(paths)

In [19]:
import numba, math

@numba.jit
def select_primary_vertex(cand, PVtx, builder):
    for i0 in range(len(cand)):
        builder.begin_list()
        for i1 in range(len(cand[i0])):
            c = cand[i0][i1]
            p = math.sqrt(c.x**2 + c.y**2 + c.z**2)
            best_idx = -1
            best_cos_theta = -1
            for i2 in range(len(PVtx[i0])):
                pv = PVtx[i0][i2]
                dx = (c.vx - pv.x)
                dy = (c.vy - pv.y)
                dz = (c.vz - pv.z)
                d = math.sqrt(dx*dx + dy*dy + dz*dz)
                cos_theta = (c.x*dx + c.y*dy + c.z*dz)/p/d

                if best_cos_theta > cos_theta: continue
                best_idx = i2
                best_cos_theta = cos_theta
            builder.begin_record()
            builder.field('dl')
            builder.real(d)
            builder.field('best_idx')
            builder.real(best_idx)
            builder.field('best_cos_theta')
            builder.real(best_cos_theta)
            builder.end_record()      
        builder.end_list()
        
    return builder


In [20]:
class MCProcessor(processor.ProcessorABC):
    def __init__(self, vertex_assignment='default'):
        Gen_Upsilon_hists = processor.dict_accumulator({
            'pt': Hist.new.Regular(100, 0, 100, name='pt', label=r"$p_{T,Gen \Upsilon}$ [GeV]").Double(),
            'eta': Hist.new.Regular(60, -4, 4, name="eta", label=r"$\eta_{Gen \Upsilon}$").Double(),
            'phi': Hist.new.Regular(60, -math.pi, math.pi, name="phi", label=r"$\phi_{Gen \Upsilon}$").Double(),
            'mass': Hist.new.Regular(100, 8.6, 11, name="mass", label=r"$m_{Gen \Upsilon}$ [GeV]").Double(),
        })

        Gen_Muon_lead_hists = processor.dict_accumulator({
            'pt': Hist.new.Regular(100, 0, 100, name='pt', label=r"$p_{T,Gen \mu}$ [GeV]").Double(),
            'eta': Hist.new.Regular(60, -2.5, 2.5, name="eta", label=r"$\eta_{Gen \mu}$").Double(),
            'phi': Hist.new.Regular(60, -math.pi, math.pi, name="phi", label=r"$\phi_{Gen \mu}$").Double(),
            'mass': Hist.new.Regular(100, 0, 1, name="mass", label=r"$m_{Gen \mu}$ [GeV]").Double(),
        })

        Gen_Muon_trail_hists = processor.dict_accumulator({
            'pt': Hist.new.Regular(100, 0, 100, name='pt', label=r"$p_{T,Gen \mu}$ [GeV]").Double(),
            'eta': Hist.new.Regular(60, -2.5, 2.5, name="eta", label=r"$\eta_{Gen \mu}$").Double(),
            'phi': Hist.new.Regular(60, -math.pi, math.pi, name="phi", label=r"$\phi_{Gen \mu}$").Double(),
            'mass': Hist.new.Regular(100, 0, 1, name="mass", label=r"$m_{Gen \mu}$ [GeV]").Double(),
        })

        Gen_Dstar_hists = {
            'pt': Hist.new.Regular(100, 0, 100, name='pt', label=r"$p_{T,Gen D^*}$ [GeV]").Double(),
            'eta': Hist.new.Regular(60, -4, 4, name="eta", label=r"$\eta_{Gen D^*}$").Double(),
            'phi': Hist.new.Regular(60, -math.pi, math.pi, name="phi", label=r"$\phi_{Gen D^*}$").Double(),
            'deltam': Hist.new.Regular(50, 0.138, 0.162, name="deltam", label=r"$\Delta m_{Gen D^*}$ [GeV]").Double(),
        }

        Gen_D0_hists = {
            'pt': Hist.new.Regular(100, 0, 100, name='pt', label=r"$p_{T,Gen D_0}$ [GeV]").Double(),
            'eta': Hist.new.Regular(60, -5, 5, name="eta", label=r"$\eta_{Gen D_0}$").Double(),
            'phi': Hist.new.Regular(60, -math.pi, math.pi, name="phi", label=r"$\phi_{Gen D_0}$").Double(),
            'mass': Hist.new.Regular(100, 1.7, 2.0, name="mass", label=r"$m_{Gen D_0}$ [GeV]").Double(),
        }

        Event_info = processor.dict_accumulator({
            'n_Evt': Hist.new.Regular(1000, -0.5, 999.5, name='n_evt', label=r"Number of Events").Int64(),
        })

        self._hists = {
            'Gen_Upsilon': {**Gen_Upsilon_hists},
            'Gen_Muon_lead': {**Gen_Muon_lead_hists},
            'Gen_Muon_trail': {**Gen_Muon_trail_hists},
            'Gen_Dstar': {**Gen_Dstar_hists},
            'Gen_D0': {**Gen_D0_hists},
            'Event_info': {**Event_info}
        }
        
        self._vertex_assignment = vertex_assignment

    @property
    def hists(self):
        return self._hists

    def process(self, events):
        if len(events) == 0:
            return self.hists

        self.hists['Event_info']['n_Evt'].fill(n_evt=len(events))

        GenPart = ak.zip({**get_vars_dict(events, gen_part_cols)}, with_name="PtEtaPhiMCandidate")

        # Gen_Muon
        GenPart_Muon = GenPart[(np.absolute(GenPart.pdgId) == 13) & (GenPart.genPartIdxMother > -1)]
        gen_ups = (GenPart_Muon.parpdgId == 553) | (GenPart_Muon.parpdgId == 100553) | (GenPart_Muon.parpdgId == 200553)
        GenPart_Muon = GenPart_Muon[gen_ups]
        GenPart_Muon = ak.combinations(GenPart_Muon, 2)
        GenPart_Muon = GenPart_Muon[GenPart_Muon.slot0.genPartIdxMother == GenPart_Muon.slot1.genPartIdxMother]

        # Gen_Upsilon
        GenPart_Ups = GenPart[(GenPart_Muon.slot0.genPartIdxMother)]

        leading_mu = (GenPart_Muon.slot0.pt > GenPart_Muon.slot1.pt)
        GenPart_Muon_lead = ak.where(leading_mu, GenPart_Muon.slot0, GenPart_Muon.slot1)
        GenPart_Muon_trail = ak.where(~leading_mu, GenPart_Muon.slot0, GenPart_Muon.slot1)

        # Gen_Ds
        GenPart_D0 = GenPart[(np.absolute(GenPart.pdgId) == 421) & (np.absolute(GenPart.parpdgId) == 413)]
        GenPart_Dstar = GenPart[GenPart_D0.genPartIdxMother]

        # Reco objs
        Dimu = ak.zip({**get_vars_dict(events, dimu_cols)}, with_name="PtEtaPhiMCandidate")
        Dimu = Dimu[Dimu.charge == 0]
        Dimu = Dimu[(Dimu.mass > 8.5) & (Dimu.mass < 11.5)]

        Muon = ak.zip({**get_vars_dict(events, muon_cols)}, with_name="PtEtaPhiMCandidate")
        Muon = ak.zip({'0': Muon[Dimu.t1muIdx], '1': Muon[Dimu.t2muIdx]})

        gen_muon_pt = (Muon.slot0.pt > 1.) & (Muon.slot1.pt > 1.)     
        Dimu = Dimu[gen_muon_pt]
        Muon = Muon[gen_muon_pt]

        gen_muon_eta = (np.absolute(Muon.slot0.eta) < 2.5) & (np.absolute(Muon.slot1.eta) < 2.5)
        Dimu = Dimu[gen_muon_eta]
        Muon = Muon[gen_muon_eta]

        D0 = ak.zip({'mass': events.D0_mass12, **get_vars_dict(events, d0_cols)}, with_name="PtEtaPhiMCandidate")
        Dstar = ak.zip({'mass': (events.Dstar_D0mass + events.Dstar_deltamr),
                        'charge': events.Dstar_pischg,
                        **get_vars_dict(events, dstar_cols)}, 
                        with_name="PtEtaPhiMCandidate")
        
        Dstar = Dstar[Dstar.simIdx > -1]
        Dstar = Dstar[~Dstar.hasMuon]
        Dstar = Dstar[(Dstar.D0mass < D0_PDG_MASS + 0.025) & (Dstar.D0mass > D0_PDG_MASS - 0.025)]
        Dstar['wrg_chg'] = (Dstar.Kchg == Dstar.pichg)
        Dstar[~Dstar.wrg_chg]
        Dstar = Dstar[np.absolute(Dstar.eta) < 2.5]
        Dstar = Dstar[np.absolute(Dstar.D0eta) < 2.5]
        Dstar = Dstar[Dstar.pt > 2.]

        process_association = ak.sum(ak.num(Dstar)) > 0 and ak.sum(ak.num(Dimu)) > 0

        if self._vertex_assignment == 'default':
            if process_association:            
                args = ak.argcartesian([Dimu.vtxIdx, Dstar.vtxIdx])
                vertex_Dimu_Dstar = ak.cartesian([Dimu.vtxIdx, Dstar.vtxIdx])
                args = args[vertex_Dimu_Dstar.slot0 == vertex_Dimu_Dstar.slot1]

                Dimu_Dstar = ak.zip({'0': Dimu[args.slot0], '1': Dstar[args.slot1]})
            
        elif self._vertex_assignment == 'custom':
            PVtx = ak.zip({**get_vars_dict(events, pvtx_cols)})

            D0_Dstar_p4 = ak.zip(
                {
                    "pt": Dstar.D0pt,
                    "eta": Dstar.D0eta,
                    "phi": Dstar.D0phi,
                    "mass": Dstar.D0mass,
                }
                , with_name='PtEtaPhiMLorentzVector')

            D0_Dstar = ak.zip({
                'x': D0_Dstar_p4.x, 
                'y': D0_Dstar_p4.y,
                'z': D0_Dstar_p4.z,
                'vx': Dstar.D0x, 
                'vy': Dstar.D0y,
                'vz': Dstar.D0z,
            })

            if process_association:
                vtx_assign = select_primary_vertex(D0_Dstar, PVtx, ak.ArrayBuilder()).snapshot()
                
                args = ak.argcartesian([Dimu.vtxIdx, vtx_assign.best_idx])
                vertex_Dimu_Dstar = ak.cartesian([Dimu.vtxIdx, vtx_assign.best_idx])
                args = args[vertex_Dimu_Dstar.slot0 == vertex_Dimu_Dstar.slot1]

                Dimu_Dstar = ak.zip({'0': Dimu[args.slot0], '1': Dstar[args.slot1]})
        
        if len(GenPart_Ups) > 0:
            for i0 in range(len(GenPart_Ups)):
                print(i0)
                print('-----GEN-----')
                for i1 in range(len(GenPart_Ups[i0])):
                    print_gen_candidate(GenPart_Ups[i0][i1])
                for i1 in range(len(GenPart_Muon_lead[i0])):
                    print('\t', end='')
                    print_gen_candidate(GenPart_Muon_lead[i0][i1])
                    print('\t', end='')
                    print_gen_candidate(GenPart_Muon_trail[i0][i1])
                for i1 in range(len(GenPart_Dstar[i0])):
                    print_gen_candidate(GenPart_Dstar[i0][i1])
                for i1 in range(len(GenPart_D0[i0])):
                    print('\t', end='')
                    print_gen_candidate(GenPart_D0[i0][i1])
                print('-----RECO-----')
                for i1 in range(len(Dimu[i0])):
                    print(f'Dimu - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Dimu[i0][i1])}, vtx: ({Dimu[i0][i1].vx:.2f}, {Dimu[i0][i1].vy:.2f}, {Dimu[i0][i1].vz:.2f}), vtxIdx: {Dimu[i0][i1].vtxIdx}')
                    print(f'\tMuon0 - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Muon.slot0[i0][i1])}, vtx: ({Muon.slot0[i0][i1].vx:.2f}, {Muon.slot0[i0][i1].vy:.2f}, {Muon.slot0[i0][i1].vz:.2f})')
                    print(f'\tMuon1 - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Muon.slot1[i0][i1])}, vtx: ({Muon.slot1[i0][i1].vx:.2f}, {Muon.slot1[i0][i1].vy:.2f}, {Muon.slot1[i0][i1].vz:.2f})')
                for i1 in range(len(Dstar[i0])):
                    print(f'Dstar - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Dstar[i0][i1])}, vtxIdx: {Dstar[i0][i1].vtxIdx}, simIdx: {Dstar[i0][i1].simIdx}')
                    print(f'\tD0 - 4-momentum (pt, eta, phi, mass): ({Dstar[i0][i1].D0pt:.2f}, {Dstar[i0][i1].D0eta:.2f}, {Dstar[i0][i1].D0phi:.2f}, {Dstar[i0][i1].D0mass:.2f}), vtx: ({Dstar[i0][i1].D0x:.2f}, {Dstar[i0][i1].D0y:.2f}, {Dstar[i0][i1].D0z:.2f}), simIdx: {Dstar[i0][i1].D0simIdx}')
                print('')
                if process_association:
                    if len(Dimu_Dstar[i0]) < 1: continue 
                    print('Found Dimu Dstar in the same vertex!')
                    for i1 in range(len(Dimu_Dstar.slot0[i0])):
                        print(f'Dimu - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Dimu_Dstar.slot0[i0][i1])}, vtx: ({Dimu_Dstar.slot0[i0][i1].vx:.2f}, {Dimu_Dstar.slot0[i0][i1].vy:.2f}, {Dimu_Dstar.slot0[i0][i1].vz:.2f}), vtxIdx: {Dimu_Dstar.slot0[i0][i1].vtxIdx}')
                        print(f'Dstar - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Dimu_Dstar.slot1[i0][i1])}, vtxIdx: {Dimu_Dstar.slot1[i0][i1].vtxIdx}, simIdx: {Dimu_Dstar.slot1[i0][i1].simIdx}')
                        print(f'\tD0 - 4-momentum (pt, eta, phi, mass): ({Dimu_Dstar.slot1[i0][i1].D0pt:.2f}, {Dimu_Dstar.slot1[i0][i1].D0eta:.2f}, {Dimu_Dstar.slot1[i0][i1].D0phi:.2f}, {Dimu_Dstar.slot1[i0][i1].D0mass:.2f}), vtx: ({Dimu_Dstar.slot1[i0][i1].D0x:.2f}, {Dimu_Dstar.slot1[i0][i1].D0y:.2f}, {Dimu_Dstar.slot1[i0][i1].D0z:.2f}), simIdx: {Dimu_Dstar.slot1[i0][i1].D0simIdx}')
                    print('')


        fill_hists(self.hists['Gen_Upsilon'], GenPart_Ups)
        fill_hists(self.hists['Gen_Muon_lead'], GenPart_Muon_lead)
        fill_hists(self.hists['Gen_Muon_trail'], GenPart_Muon_trail)
        fill_hists(self.hists['Gen_Dstar'], GenPart_Dstar, get_deltam=True)

        return self.hists

    def postprocess(self, accumulator):
        return accumulator

In [21]:
data = {"test": files[:20]}

tstart = time.time()

output = processor.run_uproot_job(
                                data, treename='Events',
                                processor_instance=MCProcessor(),
                                executor=processor.iterative_executor,
                                executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                chunksize=360000,
                                )

print(f"Process finished in: {time.time() - tstart:.2f} s")

Preprocessing:   0%|          | 0/20 [00:00<?, ?file/s]

Processing:   0%|          | 0/20 [00:00<?, ?chunk/s]

0
-----GEN-----
PDGId: 413, Id: 0, motherPDGId: 0, motherId: -999, n_daughters: 2, status: 2, 4-momentum (pt, eta, phi, mass): (9.94, -0.92, 1.43, 2.01), vtx: (-0.02, 0.07, 3.04)
PDGId: -413, Id: 1, motherPDGId: 0, motherId: -999, n_daughters: 2, status: 2, 4-momentum (pt, eta, phi, mass): (9.94, 0.92, -1.71, 2.01), vtx: (-0.02, 0.07, 3.04)
	PDGId: 421, Id: 2, motherPDGId: 413, motherId: 0, n_daughters: 2, status: 2, 4-momentum (pt, eta, phi, mass): (9.06, -0.93, 1.43, 1.86), vtx: (-0.02, 0.12, 2.99)
	PDGId: -421, Id: 4, motherPDGId: -413, motherId: 1, n_daughters: 2, status: 2, 4-momentum (pt, eta, phi, mass): (9.22, 0.93, -1.71, 1.86), vtx: (-0.02, 0.07, 3.04)
-----RECO-----
Dstar - 4-momentum (pt, eta, phi, mass): (9.89, -0.92, 1.43, 2.01), vtxIdx: 3, simIdx: 0
	D0 - 4-momentum (pt, eta, phi, mass): (9.00, -0.93, 1.43, 1.86), vtx: (-0.02, 0.12, 2.98), simIdx: 2
Dstar - 4-momentum (pt, eta, phi, mass): (9.90, 0.92, -1.71, 2.01), vtxIdx: 3, simIdx: 1
	D0 - 4-momentum (pt, eta, phi, ma

In [22]:
data = {"test": files[:20]}

tstart = time.time()

output = processor.run_uproot_job(
                                data,
                                treename='Events',
                                processor_instance=MCProcessor(vertex_assignment='custom'),
                                executor=processor.iterative_executor,
                                executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                chunksize=360000,
                                )

print(f"Process finished in: {time.time() - tstart:.2f} s")

Processing:   0%|          | 0/20 [00:00<?, ?chunk/s]

0
-----GEN-----
PDGId: 413, Id: 0, motherPDGId: 0, motherId: -999, n_daughters: 2, status: 2, 4-momentum (pt, eta, phi, mass): (9.94, -0.92, 1.43, 2.01), vtx: (-0.02, 0.07, 3.04)
PDGId: -413, Id: 1, motherPDGId: 0, motherId: -999, n_daughters: 2, status: 2, 4-momentum (pt, eta, phi, mass): (9.94, 0.92, -1.71, 2.01), vtx: (-0.02, 0.07, 3.04)
	PDGId: 421, Id: 2, motherPDGId: 413, motherId: 0, n_daughters: 2, status: 2, 4-momentum (pt, eta, phi, mass): (9.06, -0.93, 1.43, 1.86), vtx: (-0.02, 0.12, 2.99)
	PDGId: -421, Id: 4, motherPDGId: -413, motherId: 1, n_daughters: 2, status: 2, 4-momentum (pt, eta, phi, mass): (9.22, 0.93, -1.71, 1.86), vtx: (-0.02, 0.07, 3.04)
-----RECO-----
Dstar - 4-momentum (pt, eta, phi, mass): (9.89, -0.92, 1.43, 2.01), vtxIdx: 3, simIdx: 0
	D0 - 4-momentum (pt, eta, phi, mass): (9.00, -0.93, 1.43, 1.86), vtx: (-0.02, 0.12, 2.98), simIdx: 2
Dstar - 4-momentum (pt, eta, phi, mass): (9.90, 0.92, -1.71, 2.01), vtxIdx: 3, simIdx: 1
	D0 - 4-momentum (pt, eta, phi, ma

In [ ]:
x = output['Event_info']['n_evt']
np.sum(x.axes[0].centers * x.counts())

In [27]:
import uproot

ak.behavior.update(candidate.behavior)
#f = uproot.open(files[0])
f = uproot.open('/afs/cern.ch/work/k/kmotaama/public/analysis/DESY/nanoAODplus/CMSSW_10_6_20/src/NanoAOD/NanoAnalyzer/UpsilonToMuMuDstarToD0pi.root')
events = f['Events'].arrays()

In [104]:
Dimu = ak.zip({**get_vars_dict(events, dimu_cols)}, with_name="PtEtaPhiMCandidate")
Muon = ak.zip({**get_vars_dict(events, muon_cols)}, with_name="PtEtaPhiMCandidate")

Muon = ak.zip({'0': Muon[Dimu.t1muIdx], '1': Muon[Dimu.t2muIdx]})

Dstar = ak.zip({'mass': (events.Dstar_D0mass + events.Dstar_deltamr),
                        'charge': events.Dstar_pischg,
                        **get_vars_dict(events, dstar_cols)}, 
                        with_name="PtEtaPhiMCandidate")
Dstar = Dstar[~Dstar.hasMuon]
Dstar = Dstar[(Dstar.D0mass < D0_PDG_MASS + 0.025) & (Dstar.D0mass > D0_PDG_MASS - 0.025)]
Dstar['wrg_chg'] = (Dstar.Kchg == Dstar.pichg)
Dstar[~Dstar.wrg_chg]
Dstar = Dstar[np.absolute(Dstar.eta) < 2.5]
Dstar = Dstar[np.absolute(Dstar.D0eta) < 2.5]
Dstar = Dstar[Dstar.pt > 2.]

PVtx = ak.zip({**get_vars_dict(events, pvtx_cols)})
ak.num(PVtx)

D0_Dstar_p4 = ak.zip(
    {
        "pt": Dstar.D0pt,
        "eta": Dstar.D0eta,
        "phi": Dstar.D0phi,
        "mass": Dstar.D0mass,
    }
    , with_name='PtEtaPhiMLorentzVector')

D0_Dstar = ak.zip({
    'x': D0_Dstar_p4.x, 
    'y': D0_Dstar_p4.y,
    'z': D0_Dstar_p4.z,
    'vx': Dstar.D0x, 
    'vy': Dstar.D0y,
    'vz': Dstar.D0z,
}) 

In [105]:
test = select_primary_vertex(D0_Dstar, PVtx, ak.ArrayBuilder()).snapshot()

In [106]:
args = ak.argcartesian([Dimu.vtxIdx, test.best_idx])
x = ak.cartesian([Dimu.vtxIdx, test.best_idx])

mask = x.slot0 == x.slot1
args = args[mask]

args

<Array [[], [], [], [(0, ... 0, 48)], [], []] type='9 * var * (int64, int64)'>

In [107]:
Dimu_Dstar = ak.zip({'0': Dimu[args.slot0], '1': Dstar[args.slot1]})

for i0 in range(len(Dimu_Dstar.slot0)):
    if len(Dimu_Dstar.slot0[i0]) < 1: continue
    for i1 in range(len(Dimu_Dstar.slot0[i0])):
        print(f'Dimu - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Dimu_Dstar.slot0[i0][i1])}, vtx: ({Dimu_Dstar.slot0[i0][i1].vx:.2f}, {Dimu_Dstar.slot0[i0][i1].vy:.2f}, {Dimu_Dstar.slot0[i0][i1].vz:.2f}), vtxIdx: {Dimu_Dstar.slot0[i0][i1].vtxIdx}')
        print(f'Dstar - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Dimu_Dstar.slot1[i0][i1])}, vtxIdx: {Dimu_Dstar.slot1[i0][i1].vtxIdx}, simIdx: {Dimu_Dstar.slot1[i0][i1].simIdx}')
        print(f'\tD0 - 4-momentum (pt, eta, phi, mass): ({Dimu_Dstar.slot1[i0][i1].D0pt:.2f}, {Dimu_Dstar.slot1[i0][i1].D0eta:.2f}, {Dimu_Dstar.slot1[i0][i1].D0phi:.2f}, {Dimu_Dstar.slot1[i0][i1].D0mass:.2f}), vtx: ({Dimu_Dstar.slot1[i0][i1].D0x:.2f}, {Dimu_Dstar.slot1[i0][i1].D0y:.2f}, {Dimu_Dstar.slot1[i0][i1].D0z:.2f}), simIdx: {Dimu_Dstar.slot1[i0][i1].D0simIdx}')
    print('')

Dimu - 4-momentum (pt, eta, phi, mass): (4.66, -0.96, 1.09, 9.49), vtx: (-0.03, 0.07, -0.84), vtxIdx: 1
Dstar - 4-momentum (pt, eta, phi, mass): (2.59, -0.29, -2.47, 2.03), vtxIdx: 1, simIdx: -1
	D0 - 4-momentum (pt, eta, phi, mass): (2.27, -0.30, -2.48, 1.86), vtx: (-0.04, 0.06, -0.85), simIdx: -1
Dimu - 4-momentum (pt, eta, phi, mass): (4.66, -0.96, 1.09, 9.49), vtx: (-0.03, 0.07, -0.84), vtxIdx: 1
Dstar - 4-momentum (pt, eta, phi, mass): (2.02, 0.33, -2.03, 2.04), vtxIdx: 1, simIdx: -1
	D0 - 4-momentum (pt, eta, phi, mass): (1.79, 0.32, -2.01, 1.88), vtx: (-0.03, 0.07, -0.84), simIdx: -1
Dimu - 4-momentum (pt, eta, phi, mass): (5.59, -0.30, 2.00, 9.50), vtx: (-0.03, 0.07, -0.84), vtxIdx: 1
Dstar - 4-momentum (pt, eta, phi, mass): (2.59, -0.29, -2.47, 2.03), vtxIdx: 1, simIdx: -1
	D0 - 4-momentum (pt, eta, phi, mass): (2.27, -0.30, -2.48, 1.86), vtx: (-0.04, 0.06, -0.85), simIdx: -1
Dimu - 4-momentum (pt, eta, phi, mass): (5.59, -0.30, 2.00, 9.50), vtx: (-0.03, 0.07, -0.84), vtxIdx: 

In [108]:
args = ak.argcartesian([Dimu.vtxIdx, Dstar.vtxIdx])
x = ak.cartesian([Dimu.vtxIdx, Dstar.vtxIdx])

mask = x.slot0 == x.slot1
args = args[mask]

args

<Array [[], [], [], [(0, 5, ... 2), (0, 42)]] type='9 * var * (int64, int64)'>

In [109]:
Dimu_Dstar = ak.zip({'0': Dimu[args.slot0], '1': Dstar[args.slot1]})

for i0 in range(len(Dimu_Dstar.slot0)):
    if len(Dimu_Dstar.slot0[i0]) < 1: continue
    for i1 in range(len(Dimu_Dstar.slot0[i0])):
        print(f'Dimu - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Dimu_Dstar.slot0[i0][i1])}, vtx: ({Dimu_Dstar.slot0[i0][i1].vx:.2f}, {Dimu_Dstar.slot0[i0][i1].vy:.2f}, {Dimu_Dstar.slot0[i0][i1].vz:.2f}), vtxIdx: {Dimu_Dstar.slot0[i0][i1].vtxIdx}')
        print(f'Dstar - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Dimu_Dstar.slot1[i0][i1])}, vtxIdx: {Dimu_Dstar.slot1[i0][i1].vtxIdx}, simIdx: {Dimu_Dstar.slot1[i0][i1].simIdx}')
        print(f'\tD0 - 4-momentum (pt, eta, phi, mass): ({Dimu_Dstar.slot1[i0][i1].D0pt:.2f}, {Dimu_Dstar.slot1[i0][i1].D0eta:.2f}, {Dimu_Dstar.slot1[i0][i1].D0phi:.2f}, {Dimu_Dstar.slot1[i0][i1].D0mass:.2f}), vtx: ({Dimu_Dstar.slot1[i0][i1].D0x:.2f}, {Dimu_Dstar.slot1[i0][i1].D0y:.2f}, {Dimu_Dstar.slot1[i0][i1].D0z:.2f}), simIdx: {Dimu_Dstar.slot1[i0][i1].D0simIdx}')
    print('')

Dimu - 4-momentum (pt, eta, phi, mass): (4.66, -0.96, 1.09, 9.49), vtx: (-0.03, 0.07, -0.84), vtxIdx: 1
Dstar - 4-momentum (pt, eta, phi, mass): (2.49, 2.34, -2.37, 2.04), vtxIdx: 1, simIdx: -1
	D0 - 4-momentum (pt, eta, phi, mass): (2.25, 2.36, -2.35, 1.88), vtx: (0.00, 0.07, -0.92), simIdx: -1
Dimu - 4-momentum (pt, eta, phi, mass): (4.66, -0.96, 1.09, 9.49), vtx: (-0.03, 0.07, -0.84), vtxIdx: 1
Dstar - 4-momentum (pt, eta, phi, mass): (2.59, -0.29, -2.47, 2.03), vtxIdx: 1, simIdx: -1
	D0 - 4-momentum (pt, eta, phi, mass): (2.27, -0.30, -2.48, 1.86), vtx: (-0.04, 0.06, -0.85), simIdx: -1
Dimu - 4-momentum (pt, eta, phi, mass): (4.66, -0.96, 1.09, 9.49), vtx: (-0.03, 0.07, -0.84), vtxIdx: 1
Dstar - 4-momentum (pt, eta, phi, mass): (2.03, -0.63, -1.76, 2.04), vtxIdx: 1, simIdx: -1
	D0 - 4-momentum (pt, eta, phi, mass): (1.86, -0.66, -1.73, 1.87), vtx: (-0.03, 0.07, -0.84), simIdx: -1
Dimu - 4-momentum (pt, eta, phi, mass): (4.66, -0.96, 1.09, 9.49), vtx: (-0.03, 0.07, -0.84), vtxIdx: 1

In [110]:
for i0 in range(len(Dimu)):
    if len(Dimu[i0]) < 1: continue
    for i1 in range(len(Dimu[i0])):
        print(f'Dimu - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Dimu[i0][i1])}, vtx: ({Dimu[i0][i1].vx:.2f}, {Dimu[i0][i1].vy:.2f}, {Dimu[i0][i1].vz:.2f}), vtxIdx: {Dimu[i0][i1].vtxIdx}')
    for i1 in range(len(Dstar[i0])):
        print(f'Dstar - 4-momentum (pt, eta, phi, mass): {build_4mom_string(Dstar[i0][i1])}, vtxIdx: {Dstar[i0][i1].vtxIdx}, simIdx: {Dstar[i0][i1].simIdx}')
        print(f'\tD0 - 4-momentum (pt, eta, phi, mass): ({Dstar[i0][i1].D0pt:.2f}, {Dstar[i0][i1].D0eta:.2f}, {Dstar[i0][i1].D0phi:.2f}, {Dstar[i0][i1].D0mass:.2f}), vtx: ({Dstar[i0][i1].D0x:.2f}, {Dstar[i0][i1].D0y:.2f}, {Dstar[i0][i1].D0z:.2f}), simIdx: {Dstar[i0][i1].D0simIdx}')
    print('')

Dimu - 4-momentum (pt, eta, phi, mass): (6.45, 0.16, -1.54, 10.41), vtx: (-0.02, 0.07, -0.22), vtxIdx: 1
Dstar - 4-momentum (pt, eta, phi, mass): (3.00, 0.75, 1.33, 2.08), vtxIdx: 0, simIdx: -1
	D0 - 4-momentum (pt, eta, phi, mass): (2.82, 0.75, 1.36, 1.87), vtx: (-0.02, 0.06, 0.25), simIdx: -1
Dstar - 4-momentum (pt, eta, phi, mass): (2.05, 0.12, 0.96, 2.03), vtxIdx: 0, simIdx: -1
	D0 - 4-momentum (pt, eta, phi, mass): (1.78, 0.12, 0.97, 1.87), vtx: (-0.02, 0.08, 0.26), simIdx: -1

Dimu - 4-momentum (pt, eta, phi, mass): (2.11, -2.46, -2.04, 9.48), vtx: (-0.02, 0.07, 4.07), vtxIdx: 2
Dstar - 4-momentum (pt, eta, phi, mass): (3.51, -0.43, -0.85, 2.01), vtxIdx: 0, simIdx: -1
	D0 - 4-momentum (pt, eta, phi, mass): (3.32, -0.44, -0.84, 1.86), vtx: (-0.04, 0.07, 3.15), simIdx: -1
Dstar - 4-momentum (pt, eta, phi, mass): (3.46, -0.44, -0.86, 2.04), vtxIdx: 0, simIdx: -1
	D0 - 4-momentum (pt, eta, phi, mass): (3.32, -0.44, -0.84, 1.87), vtx: (-0.04, 0.07, 3.15), simIdx: -1
Dstar - 4-momentum

In [28]:
GenPart = ak.zip({**get_vars_dict(events, gen_part_cols)}, with_name="PtEtaPhiMCandidate")

In [29]:
for i0 in range(len(GenPart)):
    print(f'Event {i0}')
    for i1 in range(len(GenPart[i0])):
        print(GenPart.pdgId[i0][i1])

Event 0
413
-413
421
-421
Event 1
413
-413
421
-421
Event 2
413
-413
421
-421
Event 3
413
-413
421
-421
Event 4
413
-413
421
-421
Event 5
413
-413
421
-421
Event 6
413
-413
421
-421
Event 7
413
-413
421
-421
Event 8
413
-413
421
-421
Event 9
413
-413
421
-421
Event 10
413
-413
421
-421
Event 11
413
-413
421
-421
Event 12
413
-413
421
-421
Event 13
413
-413
421
-421
Event 14
413
-413
421
-421
Event 15
413
-413
421
-421
Event 16
413
-413
421
-421
Event 17
413
-413
421
-421
Event 18
413
-413
421
-421
Event 19
413
-413
421
-421
Event 20
413
-413
421
-421
Event 21
413
-413
421
-421
Event 22
413
-413
421
-421
Event 23
413
-413
421
-421
Event 24
413
-413
421
-421
Event 25
413
-413
421
-421
Event 26
413
-413
421
-421
Event 27
413
-413
421
-421
Event 28
413
-413
421
-421
Event 29
413
-413
421
-421
Event 30
413
-413
421
-421
Event 31
413
-413
421
-421
Event 32
413
-413
421
-421
Event 33
413
-413
421
-421
Event 34
413
-413
421
-421
Event 35
413
-413
421
-421
Event 36
413
-413
421
-421
Event 37
41

In [26]:
files[0]

'/eos/user/k/kmotaama/CRAB_UserFiles/UpsilonToMuMuDstarToD0pi/220127_103512/0000/UpsilonToMuMuDstarToD0pi_1.root'